In [197]:
import numpy as np
import pandas as pd
import copy
from collections import defaultdict

np.random.seed(1)

In [198]:
Grid = np.array([np.zeros(4) for i in range(4)])
np.random.randint(4)

1

In [202]:
class Board():
    score =0
    lose = 0
    tile = 2
    def __init__(self):
        self.grid = np.array([np.zeros(4) for i in range(4)])
        i = np.random.randint(4)
        j = np.random.randint(4)
        self.grid[i,j] = 2
        k,l = i,j

        while(i==k and l==j):
            k = np.random.randint(4)
            l = np.random.randint(4)
        self.grid[k,l] = 2

### Shift, merge, and move constitute one move ###        
    def shift(self):
        current = np.array([np.zeros(4) for i in range(4)])
        for i in range(4):
            position = 0
            for j in range(4):
                if self.grid[i][j] != 0:
                    current[i][position] = self.grid[i][j]
                    position+=1
        self.grid = current
    def merge(self):
        #shift left, merge values, shift left
        self.shift()
        for i in range(4):
            for j in range(3):
                if self.grid[i][j] == self.grid[i][j+1] and self.grid[i][j] != 0:
                    self.grid[i][j] *= 2
                    self.score+= self.grid[i][j]
                    if (self.grid[i][j]) > self.tile:
                        self.tile = self.grid[i][j]
                    self.grid[i][j+1] = 0
        self.shift()
        
    def move(self, direction):
        #Merge everything left.  Use flip and transpose for other orientations
        if direction == 'u':
            self.grid = np.transpose(self.grid)
            self.merge()
            self.grid = np.transpose(self.grid)
        elif direction == 'd':
            self.grid = np.flip(np.transpose(self.grid),1)
            self.merge()
            self.grid = np.transpose(np.flip(self.grid,1))
        elif direction == 'r':
            self.grid = np.flip(self.grid)
            self.merge()
            self.grid = np.flip(self.grid)
        else:
            self.merge()
        
### Check if the player has lost ###
    def lost(self):
        self.lose = 1
        for i in range(4):
            for j in range(4):
                if self.grid[i][j] == 0:
                    self.lose = 0 #Haven't lost yet
        for i in range(4):
            for j in range(4):
                try:
                    if (self.grid[i][j] == self.grid[i+1][j]):
                        self.lose = 0
                except:
                    pass
                try:
                    if (self.grid[i][j] == self.grid[i][j+1]):
                        self.lose = 0
                except:
                    pass
        
    def add_tile(self):
        possibilities = []
        for i in range(4):
            for j in range(4):
                if self.grid[i][j] == 0:
                    possibilities.append((i,j))
        value = np.random.choice([2,4], p=(.9,.1))
        if possibilities:
            a=possibilities[np.random.choice(len(possibilities))]
            self.grid[a] = value
    def random_game(self):
        states = []
        while (not self.lose):
            move = np.random.choice(['l','r','u','d'])
            self.move(move)
            states.append((self.grid, move))
            self.lost()
            self.add_tile()
#        print(self.grid)
#        print(self.score)
        return states
   

In [203]:
### Different game strategies here ###
def monte_carlo(game):
    def getmove(game):
        grids = []
        for i in range(10):
            grids.append(copy.deepcopy(game))
        move_scores = defaultdict(list)
        for game in grids:
            state = game.random_game()
            score = game.score
            initial_move = state[0][1]
            move_scores[initial_move].append( score)
        move = 'l'
        score = 0
        for key, value in move_scores.items():
             if (sum(value)/len(value) >score ):
                    move = key
                    score = sum(value)/len(value)
        return move
            

    states = []
    while (not game.lose):
        move = getmove(game)
        game.move(move)
        states.append((game.grid, move))
        game.lost()
        game.add_tile()
#    print(game.grid)
    print(game.score)
    print(game.tile)
    return states


In [204]:
for i in range(10):
    board = Board()
    states = monte_carlo(board)

352.0
32.0
14536.0
1024.0
7140.0
512.0
1784.0
128.0
2224.0
256.0
5228.0
512.0
5496.0
512.0
9396.0
512.0
11160.0
1024.0
1508.0
128.0


In [193]:
board = Board()
board.random_game()
board.score

1068.0